In [1]:
from google.colab import drive
import os
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/DS8009 Deep Learning/Week 10')
os.listdir()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


['turTest2KForStudent.txt',
 'en1k.txt',
 'turTrain6K.txt',
 'fra9k.txt',
 'week10.pptx',
 'turTrain6K.gdoc',
 'turTest2KForStudent.gdoc',
 '8009 Week10 lab .ipynb',
 's2s.h5',
 'week10.gslides',
 'Copy of 8009 Week10 lab .ipynb',
 'result_assign4Leo.txt',
 'result_assign4Leo.gdoc',
 'result_assign4_bidirectionalLeo.txt',
 'turTest2K.txt',
 'Assignment4_Leo .ipynb']

https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py

**References**
- [Sequence to Sequence Learning with Neural Networks
   ](https://arxiv.org/abs/1409.3215)
- [Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    ](https://arxiv.org/abs/1406.1078)
    
    
'''

Data set


http://www.manythings.org/anki/

# Assignment 4 Begins

In [0]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense, GRU, Embedding, Bidirectional,Input,Concatenate
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 5  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'turTrain6K.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

Number of samples: 6000
Number of unique input tokens: 30
Number of unique output tokens: 31
Max sequence length for inputs: 19
Max sequence length for outputs: 16


In [0]:
#reading my test data
test_text = []
test_char = set()
#setting up our test data into a list
with open('turTest2KForStudent.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    #test_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    test_text.append(line)
    for line in test_text:
      for char in line:
        if char not in test_char:
          test_char.add(char)
          
test_char = sorted(list(test_char))
num_encoder2_tokens = len(test_char)
max_encoder2_seq_length = max([len(txt) for txt in test_text])

print('Number of samples:', len(test_text))
print('Number of unique input tokens:', num_encoder2_tokens)
print('Max sequence length for inputs:', max_encoder2_seq_length)

test_token_index = dict(
    [(char, i) for i, char in enumerate(test_char)])

Number of samples: 2000
Number of unique input tokens: 30
Max sequence length for inputs: 18


In [0]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


In [0]:
my_text = 'masallar'
placeholder = np.zeros((1,len(my_text)+10,num_encoder_tokens))

for i, char in enumerate(my_text):
    print(i,char, input_token_index[char])
    placeholder[0,i,input_token_index[char]] = 1

0 m 12
1 a 0
2 s 17
3 a 0
4 l 11
5 l 11
6 a 0
7 r 16


In [0]:
#encoding testing data

encoder_test_text = np.zeros(
    (len(test_text), max_encoder2_seq_length, num_encoder2_tokens),
    dtype='float32')

for i, (test_text) in enumerate(test_text):
    for t, char in enumerate(test_text):
        encoder_test_text[i, t, test_token_index[char]] = 1.

In [0]:
#basic LSTM encoder/decoder
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [0]:
#Define an input sequence and process it bidirectional LSTM

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = Bidirectional(LSTM(latent_dim, return_state=True))                      
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]


# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim*2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [0]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='RMSprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=100,
          validation_split=0.1)

Train on 5400 samples, validate on 600 samples
Epoch 1/100
5400/5400 [==============================] - 37s 7ms/step - loss: 1.0400 - val_loss: 1.0322
Epoch 2/100
5400/5400 [==============================] - 33s 6ms/step - loss: 0.8943 - val_loss: 0.8630
Epoch 3/100
5400/5400 [==============================] - 32s 6ms/step - loss: 0.7544 - val_loss: 0.6617
Epoch 4/100
5400/5400 [==============================] - 33s 6ms/step - loss: 0.6115 - val_loss: 0.5844
Epoch 5/100
5400/5400 [==============================] - 33s 6ms/step - loss: 0.4936 - val_loss: 0.4829
Epoch 6/100
5400/5400 [==============================] - 33s 6ms/step - loss: 0.3932 - val_loss: 0.3578
Epoch 7/100
5400/5400 [==============================] - 32s 6ms/step - loss: 0.3096 - val_loss: 0.3037
Epoch 8/100
5400/5400 [==============================] - 32s 6ms/step - loss: 0.2419 - val_loss: 0.2608
Epoch 9/100
5400/5400 [==============================] - 32s 6ms/step - loss: 0.1901 - val_loss: 0.2566
Epoch 10/100
5400

In [0]:
#LSTM Encoder
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [0]:
#Bidirectional LSTM Encoder
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim*2,))
decoder_state_input_c = Input(shape=(latent_dim*2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [0]:
seq_index =1
input_texts[seq_index]
encoder_input_data[seq_index: seq_index + 1].shape
#encoder_input_dat
#decode_sequence()

In [0]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: şarabın
Decoded sentence: şarap

-
Input sentence: ret
Decoded sentence: red

-
Input sentence: masallar
Decoded sentence: masal

-
Input sentence: konserlere
Decoded sentence: konser

-
Input sentence: armağanı
Decoded sentence: armağan

-
Input sentence: sıkıyordu
Decoded sentence: sık

-
Input sentence: bulguların
Decoded sentence: bulgu

-
Input sentence: yarına
Decoded sentence: yarın

-
Input sentence: sandıklar
Decoded sentence: sandık

-
Input sentence: modelini
Decoded sentence: model



In [0]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_test_text[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', test_text[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: bıyıkları
Decoded sentence: bıyık

-
Input sentence: ayrımında
Decoded sentence: ayrım

-
Input sentence: hazırlığı
Decoded sentence: hazırlık

-
Input sentence: seviyorum
Decoded sentence: sev

-
Input sentence: genişlikte
Decoded sentence: geniş

-
Input sentence: hazırlayın
Decoded sentence: hazırla

-
Input sentence: olmam
Decoded sentence: ol

-
Input sentence: tavrıyla
Decoded sentence: tavrı

-
Input sentence: düşünmüyorsun
Decoded sentence: düşün

-
Input sentence: belgelere
Decoded sentence: belge



In [0]:
result = []
for seq_index in range(len(test_text)):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_test_text[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    result.append(decoded_sentence)

In [0]:
#writing the results onto a .txt file
with open('result_assign4_bidirectionalLeo.txt', 'w') as f:
    for item in result:
        f.write("%s" % item)

**Insert Accuracy**

In [0]:
result2 = list(zip(test_text,result))
result2[0][1]

In [2]:
clean_target = []

for i in range(len(target_texts)):
  clean = target_texts[i].strip()
  clean_target.append(clean)

clean_target

NameError: ignored

In [0]:
clean_result = []

for i in range(len(result)):
  clean = result[i].strip()
  clean_result.append(clean)

clean_result

['bıyık',
 'ayrım',
 'hazırlık',
 'sev',
 'geniş',
 'hazırla',
 'ol',
 'tavrı',
 'düşün',
 'belge',
 'yavrum',
 'bina',
 'öneri',
 'can',
 'sevgili',
 'taban',
 'ay',
 'okul',
 'biz',
 'bağla',
 'akıl',
 'adale',
 'zaman',
 'kapı',
 'aile',
 'zıkım',
 'dur',
 'ilgilen',
 'baş',
 'silaf',
 'ense',
 'ver',
 'aydınlat',
 'değişim',
 'heyben',
 'kafa',
 'karı',
 'anal',
 'baş',
 'düş',
 'gerek',
 'dil',
 'damla',
 'yön',
 'anla',
 'yarat',
 'milletekçil',
 'ara',
 'kal',
 'amara',
 'hava',
 'bakan',
 'kum',
 'dal',
 'ziyaret',
 'cins',
 'tarih',
 'hepsi',
 'şem',
 'iç',
 'git',
 'azal',
 'aşk',
 'getir',
 'yet',
 'ağız',
 'ver',
 'performans',
 'dizi',
 'kitap',
 'tezgat',
 'alet',
 'bulgu',
 'hal',
 'sahahlık',
 'öğe',
 'zih',
 'buyur',
 'mağaza',
 'cumarbaş',
 'tut',
 'gel',
 'getir',
 'mücadele',
 'baba',
 'damar',
 'kullanım',
 'akıl',
 'hayret',
 'kanal',
 'buyur',
 'ol',
 'alışveriş',
 'dur',
 'hayat',
 'hazırla',
 'miktar',
 'koku',
 'diz',
 'kupon',
 'kulak',
 'uyuş',
 'saç',
 'kam

In [0]:
counter = 0 
for word in clean_result:
  if word in clean_target:
    counter +=1
    
counter

1609

In [0]:
with open('result_assign4Leo.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
lines

In [0]:
counter = 0 
for word in lines:
  if word in clean_target:
    counter +=1
    
counter

1564

In [0]:
result_assign4_bidirectionalLeo.txt